# 1. Preprocessing of EXTOD 101 for ML 

The goal of this notebook is to prepare the EXTOD 101 data for machine learning - predicting euglycemia during and around exercise.

EXTOD (Exercise in Type One Diabetes) 101 was a trial looking into the glucose control of indivuals with T1D training for the Swansea half marathon. There was around 35 people with 8 weeks worth of FreeStyle Libre data and exercise diaries recording their exercise bouts. There is also demographic data (including age, BMI, years since diagnosis) and lab data (including HbA1c, Cpeptide).

### Objectives
1. Clean and combine the exercise diaries
2. Preprocess demographic data
3. Preprcoess lab data
4. Prepare FGM data

## 1.0. Import packages

In [38]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
import datetime
from functools import reduce
import os
import preprocess_helper
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## 1.1. Clean and combine exercise diaries

### 1.1.1. Load diaries

In [39]:
# Set directories for data
directory_cgm = '../../Data/raw_data/Libre data/'
directory_diaries = '../../Data/raw_data/Diary data/'
directory_demo_lab = '../../Data/raw_data/'

### 1.1.2. Combine dataframes

In [40]:
# Empty dataframe
ex_df = pd.DataFrame()
# Loop through all exercise diaries
for filename in os.listdir(directory_diaries):
    filepath = directory_diaries + filename
    df = pd.read_excel(filepath, sheet_name=0).append(pd.read_excel(filepath, sheet_name=1))
    # Split the two side-by-side dfs and join them
    df = preprocess_helper.clean_diaries(df)
    # Set id as first 4 characters of 
    df['ID'] = filename[:4]
    # Fix any dodgy times
    df['What time']= df['What time'].apply(lambda x: preprocess_helper.check_time(x))
    # Drop any instances that don't have datetime data
    df.dropna(subset=['What time', 'Date', 'how long'], inplace=True)
    # Combine date and time
    df['start_datetime'] = df.apply(lambda row: pd.Timestamp.combine(row['Date'], row['What time']), axis=1)
    # Convert duration to a float
    #df['how long'] = df['how long'].apply(lambda x: float(x))
    # Add duration to start time to get finish time
    df['finish_datetime'] = df.apply(lambda x: x['start_datetime']
                                     + datetime.timedelta(minutes=x['how long']), axis=1)
    # Drop useless columns and reset index
    df = df.drop(columns=['Date', 'What time', 'how long', 'Exercise'])
    df.reset_index(drop=True,inplace=True)
    # Append to global dataframe
    ex_df = ex_df.append(df)

In [41]:
ex_df.shape[0]

779

In [42]:
ex_df.head()

,Type of exericse,Intensity,Glucose before,Glucose After,low during,high during,Fast acting insulin change,basel Insulin change,Take extra carb?,how much,Last meal,comment,ID,start_datetime,finish_datetime
0,walk,17.0,11.0,13.0,no,no,yes decrease 30%,yes decrease 70% 8:30-17:00,yes,15 grams every 45 minutes,NaN,NaN,3039,2018-05-13 09:00:00,2018-05-13 16:00:00
1,run,12.0,12.0,7.0,no,no,no,yes decrease 90% 8:15-9:20,yes,20 grams at start,NaN,NaN,3039,2018-05-19 09:00:00,2018-05-19 09:28:00
2,run,13.0,13.0,7.0,no,no,no,yes decrease 90% 17:00-19:00,yes,20 grams every 30 minutes,NaN,NaN,3039,2018-05-22 18:00:00,2018-05-22 19:17:00
3,run,17.0,11.0,13.0,no,no,yes decrease 60%,yes decrease 90% 9:00-11:00,yes,15 grams every 30 minutes,NaN,NaN,3039,2018-05-28 09:30:00,2018-05-28 10:33:00
4,weights,15.0,9.0,5.0,yes,no,no,yes decrease 50% 16:30-18:00,yes,25 mg at start and 30 minutes in,NaN,NaN,3039,2018-05-31 17:00:00,2018-05-31 18:30:00


In [43]:
# Select useful columns and rename them
cols_to_choose = ['ID', 'start_datetime', 'finish_datetime', 'Intensity',
                  'Type of exericse', 'Glucose before', 'Glucose After']
diaries = ex_df.loc[:,cols_to_choose]
diaries.columns = ['ID', 'start_datetime', 'finish_datetime', 'intensity',
                  'type_of_exercise', 'starting_glucose', 'finishing_glucose']

In [44]:
# Set start and finish to datetime
diaries.start_datetime = pd.to_datetime(diaries.start_datetime)
diaries.finish_datetime = pd.to_datetime(diaries.finish_datetime)

### 1.1.3. Sort out dates

In [45]:
# Get breakdown for start datetime
diaries = preprocess_helper.date_preprocessing(diaries, 'start_datetime', False, True, True)
# Calculate duration
diaries['duration'] = (diaries['finish_datetime'] - diaries['start_datetime']).apply(lambda x: x.total_seconds()/60)

In [46]:
diaries.head()

,ID,start_datetime,finish_datetime,intensity,type_of_exercise,starting_glucose,finishing_glucose,month,day,day_of_week,time_of_day,duration
0,3039,2018-05-13 09:00:00,2018-05-13 16:00:00,17.0,walk,11.0,13.0,5,13,6,morning,420.0
1,3039,2018-05-19 09:00:00,2018-05-19 09:28:00,12.0,run,12.0,7.0,5,19,5,morning,28.0
2,3039,2018-05-22 18:00:00,2018-05-22 19:17:00,13.0,run,13.0,7.0,5,22,1,evening,77.0
3,3039,2018-05-28 09:30:00,2018-05-28 10:33:00,17.0,run,11.0,13.0,5,28,0,morning,63.0
4,3039,2018-05-31 17:00:00,2018-05-31 18:30:00,15.0,weights,9.0,5.0,5,31,3,afternoon,90.0


### 1.1.4. Categorise exercise types

In [47]:
# Rename all the dodgy words
diaries['type_of_exercise'].replace({'Cycle': 'cycle', 'walking': 'walk', 
                                        'hike':'walk', 'pliates': 'pilates',
                                       'rin': 'run', 'Run':'run', 'ron':'run',
                                       'run ': 'run'}, inplace=True)

In [48]:
# Number of bouts before
diaries.shape[0]

779

In [49]:
diaries['type_of_exercise'].value_counts()

run                  464
walk                  78
gym                   58
cycle                 52
exercise class        27
yoga                  24
golf                  16
pilates               13
rock climbing          5
cycle-run-cycle        5
HIT                    5
badminton              3
cross trainer          3
interval training      2
circuits               2
core                   2
painting               2
boxing                 2
run/gym                2
dance                  1
swim                   1
housework              1
boot camp              1
2.6                    1
paddle boarding        1
step class             1
surfing                1
Trampolining           1
manual work            1
tough mudder           1
Barre                  1
weights                1
gardening              1
Name: type_of_exercise, dtype: int64

In [50]:
# Drop the non-exercise activities
diaries = diaries.loc[~diaries['type_of_exercise'].isin(['housework', 'gardening', 'manual work'])]

In [51]:
# Number of bouts after
diaries.shape[0]

776

In [52]:
# Divide the exercise types into aerobic, anaerobic and mixed
dict_exercise_type = {'run': 'aer', 'walk': 'aer', 'gym': 'mix', 'cycle': 'aer',
                      'swim': 'aer', 'exercise class': 'mix', 'yoga': 'ana', 'pilates': 'ana',
                     'golf': 'aer', 'cycle-run-cycle': 'aer', 'HIT': 'ana', 
                     'rock climbing': 'ana', 'core': 'ana', 'badminton': 'aer',
                     'cross trainer': 'aer', 'painting': 'mix', 'boxing': 'mix',
                     'circuits': 'mix', 'Barre': 'ana', 'Trampolining': 'aer', '2.6':np.nan,
                     2.6: np.nan, 'interval training': 'mix', 'dance': 'aer',
                     'boot camp': 'mix', 'surfing': 'mix', 'paddle boarding': 'mix',
                      'step class': 'aer','tough mudder': 'mix', 'run/gym':'mix', 'weights':'ana'
                  }
diaries['form_of_exercise'] = diaries['type_of_exercise'].replace(dict_exercise_type)

In [53]:
diaries.form_of_exercise.value_counts()

aer    625
mix     99
ana     51
Name: form_of_exercise, dtype: int64

## 1.2. Preprocess demographics data

In [54]:
# Upload demographic data
df_demo = pd.read_excel(directory_demo_lab+'Summary patients data EDITED.xlsx', sheet_name='Demographics')
# Remove ones that don't have data
df_demo = df_demo.loc[df_demo['Questionnaire 1'] == 'Yes']

### 1.2.1. Calculate BMI

In [55]:
# Fill null values in order to manipulate columns
df_demo.loc[:,['Metres', 'Centimetres', 'Feet ', 'Inches']].fillna(0, inplace=True)

In [56]:
# Height recorded in different metrics so need to be sorted out (bloody medics)
df_demo['height_cm'] = df_demo['Metres']*100 + df_demo['Centimetres']
df_demo['height_inch'] = df_demo['Feet ']*12 + df_demo['Inches']
df_demo['height_inch'] = df_demo['height_inch']*2.54
df_demo['height_inch'].replace(0, np.nan, inplace=True)
df_demo['height'] = df_demo['height_inch'].fillna(df_demo['height_cm'])

In [57]:
# Ditto for weight
df_demo['weight_pounds'] = (df_demo.Stone*14 + df_demo.Pounds)*0.454
df_demo.weight_pounds.replace(0, np.nan, inplace=True)
df_demo['weight'] = df_demo.weight_pounds.fillna(df_demo.Kilograms)

In [58]:
# Calculate bmi
df_demo['bmi'] = df_demo['weight']/(df_demo['height']/100*df_demo['height']/100)

In [59]:
# Drop unneeded columns
df_demo.drop(columns = ['Stone', 'Pounds', 'Kilograms', 'Feet ', 'Inches',
                        'Metres', 'Centimetres', 'height_cm', 'height_inch',
                        'Questionnaire 1', 'date entered study'], inplace=True)

### 1.2.2. Change col names and drop redundant cols

In [60]:
# Rename cols
column_names = {'Participant No.': 'ID', 'Age (years)': 'age', 
                'Sex (1=m, 2=F)': 'sex', 'Diagnosed Month': 'diagnosis_month', 
                'Diagnosed Year': 'diagnosis_year', 'length of diabetes years': 
                'years_since_diagnosis', 'date diagnosed with Diabetes': 
                'diagnosis_date','Treat Diabetes 1= MDI, 2 Pump': 
                'insulin_administration', 'Insulin in Pump': 'insulin_in_pump', 
                'fast acting insulin': 'fast_insulin', 'Lon acting Insulin':
                'long_insulin', 'Once or Twice': 'once_or_twice',
                'Total BI': 'total_bi', 'Ratios (1= yes, 2 = no)': 'ratios',
                'Correction Factor': 'correction_factor', 
                'Eyes_1 1= yes, 2 No': 'eyes_1',
               }
df_demo.rename(column_names, axis=1, inplace=True)

In [61]:
# Drop cols
columns_drop = ['DOB', 'diagnosis_date', 'insulin_in_pump', 'fast_insulin', 
                'long_insulin']
df_demo.drop(columns=columns_drop, inplace=True)

In [62]:
# Reset index
df_demo.reset_index(inplace=True, drop=True)

In [63]:
# Replace binary with categorical
df_demo.sex.replace({1:'male', 2:'female'}, inplace=True)
df_demo.insulin_administration.replace({1:'mdi', 2:'pump'}, inplace=True)

In [64]:
df_demo_cut = df_demo.loc[:,['ID', 'age', 'sex', 'years_since_diagnosis', 'insulin_administration', 'bmi']]

In [65]:
df_demo_cut

,ID,age,sex,years_since_diagnosis,insulin_administration,bmi
0,3001,41.615332,female,10.064339,pump,23.112472
1,3002,48.008214,female,17.065024,mdi,23.353250
2,3003,52.049281,male,48.980151,pump,23.205444
3,3004,30.047912,male,21.560575,mdi,24.957697
4,3005,47.775496,male,4.065708,mdi,26.523164
5,3006,45.976728,female,36.232717,pump,24.132231
6,3007,45.229295,male,3.898700,pump,24.722222
7,3008,51.080082,male,3.561944,pump,27.471707
8,3009,80.158795,male,40.813142,mdi,23.357431
9,3010,49.702943,male,0.479124,mdi,22.914354


## 1.3. Preprocess lab data

In [66]:
# Upload the lab data
df_lab = pd.read_excel(directory_demo_lab + 'Summary patients data EDITED.xlsx',
                       sheet_name='Lab results')

In [67]:
# Drop unnecessary columns
to_drop = ['Date of Receipt','Urine C-peptide/Creatinine Ratio (nmol/mmol)', 
           'Date of Receipt', 'Date of Receipt2', 'Date of Receipt3',
           'HBAIC Sample ID', 'Barcode Set', 'Urine Sample ID',
           'Plasma Sample ID']
df_lab.drop(columns=to_drop, inplace=True)
# Rename columns
df_lab.columns = ['ID', 'visit', 'urine_cpep', 'urine_creat', 'hba1c', 'chol',
                 'cpep', 'h_index', 'hdl', 'ldl', 'nhdl', 'trig']

In [68]:
# Replace missing first visits with second visit results
df_lab = df_lab.sort_values(['ID', 'visit']).reset_index(drop=True).groupby('ID').apply(preprocess_helper.get_lab_results)
# Select only first visit results
df_lab = df_lab[df_lab['visit']==1]

In [69]:
# Merge demographic and lab data
demo_lab = df_demo_cut.merge(df_lab, how='outer', on='ID').drop(columns='visit')

## 1.4. Preprocess FGM data

In [70]:
directory= '../../Data/raw_data/Libre data'

In [71]:
# Declare empty dataframe
cgm_total= pd.DataFrame()
#loop through every filename in the directory
for filename in os.listdir(directory):
    print(filename)
    # set filepath for each file in directory
    filepath = directory+'/'+filename
    # Get the ID from the filename by selecting last 4 characters
    ID = int(filename[:4])
    # Upload cgm dataset for the file
    df = pd.read_csv(filepath, header=2)
    # Set the timestamp to datetime and sort
    df['Meter Timestamp'] = pd.to_datetime(df['Meter Timestamp'], dayfirst=True)
    df = df.sort_values('Meter Timestamp', ascending=True)
    # Set a column 'ID' in the dataframe as the ID so it can be identified later on
    df['ID'] = ID
    cgm_total = cgm_total.append(df)

3016 2.csv
3034 2.csv
3026 2.csv
3031 2.csv
3032 2.csv
3046 2.csv
3020 2.csv
3023 2.csv
3001 2.csv
3045 2.csv
3039 2.csv
3010 2.csv
3019 2.csv
3047 2.csv
3041 2.csv
3044 2.csv
3011 2.csv
3015 2.csv
3004 2.csv
3005 2.csv
3013 2.csv
3038 2.csv
3036 2.csv
3027 2.csv
3003 2.csv
3007 2.csv
3021 2.csv
3024 2.csv
3040 2.csv
3012 2.csv
3014 2.csv
3008 2.csv
3002 2.csv
3037 2.csv
3033 2.csv
3018 2.csv
3043 2.csv
3017 2.csv
3009 2.csv
3030 2.csv
3028 2.csv
3035 2.csv


In [72]:
cgm_data = cgm_total.loc[:,['ID', 'Meter Timestamp','Historic Glucose(mmol/L)','Scan Glucose(mmol/L)',]]
cgm_data.columns = ['ID', 'time', 'glc', 'scan_glc']
cgm_data.dropna(how='all', subset=['glc', 'scan_glc'], inplace=True)
cgm_data.reset_index(inplace=True, drop=True)
cgm_data = cgm_data[~cgm_data[['time', 'ID']].duplicated(keep='first')]

In [73]:
cgm_data.head()

,ID,time,glc,scan_glc
0,3016,2018-06-05 14:46:00,10.3,NaN
1,3016,2018-06-05 14:55:00,NaN,9.8
2,3016,2018-06-05 15:01:00,9.8,NaN
3,3016,2018-06-05 15:03:00,NaN,10.1
4,3016,2018-06-05 15:07:00,NaN,9.9


## 1.5. Save files

In [74]:
df_directory = '../../Data/tidy_data/'
diaries.to_csv(df_directory+'extod_101_diaries.csv', index=False)
cgm_data.to_csv(df_directory+'extod_101_cgm.csv', index=False)
demo_lab.to_csv(df_directory+'extod_101_demo_lab.csv', index=False)